In [1]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras import layers,Input
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

In [2]:
from tensorflow.keras.datasets import mnist
mnist=mnist.load_data()

(x_train,y_train),(x_test,y_test) = mnist

x_train = x_train.reshape((60000,28,28,1))
x_train = x_train.astype('float32') / 255

x_test = x_test.reshape((10000,28,28,1))
x_test = x_test.astype('float32') / 255

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [3]:
def get_cnn_model_1():
    
    input_tensor = Input(shape=(28,28,1))

    x1 = layers.Conv2D(32,3)(input_tensor)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Activation('relu')(x1)
    x1 = layers.Conv2D(64,3)(x1)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Activation('relu')(x1)
    x1 = layers.Conv2D(128,3)(x1)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Activation('relu')(x1)
    x1 = layers.MaxPool2D(2,strides=2)(x1)
    x1 = layers.Conv2D(256,3)(x1)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Activation('relu')(x1)
    x1 = layers.MaxPool2D(2,strides=2)(x1)

    x1 = layers.Flatten()(x1)

    x1 = layers.Dense(64)(x1)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Activation('relu')(x1)

    output = layers.Dense(10,activation='softmax')(x1)

    model = Model(input_tensor,output)
    
    model.compile(loss='categorical_crossentropy',
             optimizer= optimizers.RMSprop(lr=0.01,decay=0.9),
              metrics=['accuracy'])
    return model

In [4]:
def get_cnn_model_2():
    
    input_tensor = Input(shape=(28,28,1))

    x1 = layers.Conv2D(32,3)(input_tensor)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Activation('relu')(x1)
    x1 = layers.Conv2D(32,3)(x1)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Activation('relu')(x1)
    x1 = layers.MaxPool2D(2,strides=2)(x1)

    x1 = layers.Conv2D(64,3)(x1)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Activation('relu')(x1)
    x1 = layers.Conv2D(128,3)(x1)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Activation('relu')(x1)
    x1 = layers.MaxPool2D(2,strides=2)(x1)

    x1 = layers.Flatten()(x1)

    x1 = layers.Dense(64)(x1)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Activation('relu')(x1)

    output = layers.Dense(10,activation='softmax')(x1)

    model = Model(input_tensor,output)
    
    model.compile(loss='categorical_crossentropy',
             optimizer= optimizers.RMSprop(lr=0.01,decay=0.9),
              metrics=['accuracy'])
    return model

# training


In [5]:
earlystop = EarlyStopping(monitor='acc',
                         patience=3)
checkpoint = ModelCheckpoint(filepath='weight_dir/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
                             monitor='val_loss', 
                             save_weights_only=True,
                             verbose=1,
                             save_best_only=True, 
                             period=1)

In [7]:
model_1 = get_cnn_model_1()
model_1.optimizer.lr=0.01
history = model_1.fit(x_train,y_train,epochs=50,batch_size=128,callbacks=[earlystop,checkpoint],
                   validation_data=(x_test,y_test)
                   )

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
 1920/60000 [..............................] - ETA: 20:29 - loss: 0.9363 - acc: 0.7234

KeyboardInterrupt: 

In [ ]:
model_2 = get_cnn_model_2()
model_2.optimizer.lr=0.01
history = model_2.fit(x_train,y_train,epochs=50,batch_size=128,callbacks=[earlystop,checkpoint],
                   validation_data=(x_test,y_test)
                   )

# predicte


In [ ]:
model_1 = get_cnn_model_1()
model_1.load_weights('weight_dir_5/weights.05-0.03.hdf5')
model_2 = get_cnn_model_2()
model_2.load_weights('weight_dir_5/weights.05-0.03.hdf5')

In [ ]:
prediction_1 = model_1.predict(x_test,batch_size=128,verbose=1)
prediction_2 = model_2.predict(x_test,batch_size=128,verbose=1)

In [ ]:
prediction = 0.5*prediction_1 + 0.5*prediction_2

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),prediction],axis = 1)
submission.to_csv("cnn_mnist_datagen_2.csv",index=False)